In [1]:
import requests

In [2]:
import pandas as pd
import numpy as np 
import geopandas as gpd 

### Scrape website for rebuild directory data 

In [3]:
from urllib.request import Request, urlopen
url = "https://www.phila.gov/programs/rebuild/project-sites/"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
web_byte = urlopen(req).read()
webpage = web_byte.decode('utf-8')


In [4]:
from bs4 import BeautifulSoup

In [34]:
soup = BeautifulSoup(webpage, 'html.parser')
table = soup.find('table')
rsite = table.find_all('tr')

In [35]:
sites = []
for r in rsite: 
    sites.append(r.text)

In [36]:
df = pd.DataFrame(sites)
df = df[0].str.split("\n", n =4, expand = True)
df = df.drop([0,4], axis = 1)
df= df.rename(columns=df.iloc[0]).drop(df.index[0])

In [37]:
address = df['Address'].str.split(',' ,n=1, expand = True)
df['Address'] = address[0]

In [38]:
scraped = df.copy()
scraped

,Project site,Address,Status
1,Capitolo Playground,900 Federal St.,"Phase I completed, under construction"
2,Cecil B. Moore Recreation Center,2551 N. 22nd St.,Community engagement and design
3,Disston Recreation Center,4423 Longshore Ave.,Under construction
4,East Poplar,820 N. 8th St.,Community engagement and design
5,Fishtown Recreation Center,1202 E. Montgomery Ave.,"Phase I completed, construction coming soon"
...,...,...,...
58,Vogt Recreation Center,4131 Unruh Ave.,"Time-sensitive improvements, more to come"
59,Waterloo Playground,2502-12 N. Howard St.,Community engagement and design
60,West Mill Creek Playground,5100 Parrish St.,"Time-sensitive improvements, more to come"
61,Wynnefield Library,5325 Overbrook Ave.,"Time-sensitive improvements, more to come"


### Load rebuild data 

In [10]:
# path = 'E:/August Data'

In [11]:
# rebuild = gpd.GeoDataFrame(gpd.read_file(path +'/facility/rebuild_sites2.shp'))
# rebuild = rebuild.rename(columns = {'Copy_of__7' : 'status'})  

In [12]:
import esri2gpd

In [13]:
rebuild = esri2gpd.get("https://services.arcgis.com/fLeGjb7u4uXqeF9q/arcgis/rest/services/Rebuild_Sites/FeatureServer/0")

In [14]:
rebuild

,geometry,OBJECTID,ASSET_ID,ASSET_NAME,ASSET_ADDR,SITE_NAME,SITE_CODE,SHAREBLDG_ASSETID,SHAREBLDG_NOTE,ASSET_GROUP1,...,RebuildSitesFinalTable_20_14171,RebuildSitesFinalTable_20_14172,RebuildSitesFinalTable_20_14173,RebuildSitesFinalTable_20_14174,NC_PERMITS_CLASSES,GlobalID,CreationDate,Creator,EditDate,Editor
0,POINT (-75.17056355643487 39.94467958395135),1,560.0,Rodman Street,1703-05 RODMAN ST,Rodman Street,16525E,None,None,A11,...,1.0,1.0,2.0,1.0,2.0,87d79729-8531-4b08-90a0-b6c1514de3e7,1506614782385,maps.phl.data,1511359963220,David.Gould@phila.gov_phl
1,POINT (-75.1235895418245 39.98385501673952),2,787.0,"Black, Coyle & Mcbride Playground Building",2150-72 E HUNTINGDON ST,"Black, Coyle & Mcbride Playground",16127E,None,None,A11,...,2.0,2.0,2.0,4.0,1.0,9bd29ddc-c6ef-45ae-8ddc-9cc8708b6b6e,1506614782385,maps.phl.data,1511533941032,David.Gould@phila.gov_phl
2,POINT (-75.10247075585168 40.0041219393449),3,781.0,Merritt Square Park,1321 STEINBER ST,Merritt Square Park,16427E,None,None,A11,...,2.0,2.0,2.0,2.0,1.0,2c50daa7-d1bf-41b7-87da-6614f748b287,1506614782385,maps.phl.data,1511533944884,David.Gould@phila.gov_phl
3,POINT (-75.05410302029202 40.05023911603968),4,236.0,Bradford Park,2400 FAUNCE ST,Bradford Park,17413E,None,None,A11,...,3.0,1.0,1.0,4.0,1.0,d29fb76b-5458-42b8-9380-56c8df439d4e,1506614782385,maps.phl.data,1511359962125,David.Gould@phila.gov_phl
4,POINT (-75.0329017330388 40.01988808374374),5,3089.0,Tacony Access/Launch,7071R MILNOR ST,Princeton Avenue Boat Launch,None,None,None,A11,...,3.0,4.0,1.0,5.0,1.0,205d1c42-ad7e-443f-be19-fbefc82bcbda,1506614782385,maps.phl.data,1511533945455,David.Gould@phila.gov_phl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,POINT (-75.15255568538679 40.02636036273807),570,771.0,1655 W Mentor Street,1655 W Mentor Street,1655 W Mentor Street,16425E,NaN,None,None,...,4.0,4.0,1.0,1.0,1.0,0543a462-a56c-45e2-b918-4999a11e7deb,1506614782385,maps.phl.data,1506614782385,maps.phl.data
570,POINT (-75.12370634582979 39.98220611746277),571,742.0,Sepviva Street,2516-20 Sepviva Street,Sepviva Street,16541E,NaN,None,None,...,2.0,2.0,2.0,4.0,1.0,d0775ef6-6cff-40b1-97fc-4f56a37b75eb,1506614782385,maps.phl.data,1506614782385,maps.phl.data
571,POINT (-75.20063966653758 39.9415652495301),572,9999.0,FLP Sorting Facility (NEW),3401 Grays Ferry Ave,FLP Sorting Facility (NEW),10054I,NaN,None,None,...,4.0,3.0,1.0,1.0,2.0,99117632-07da-4daa-a617-97d869ddb5bc,1506614782385,maps.phl.data,1506614782385,maps.phl.data
572,POINT (-75.20820632890269 39.99629867168584),573,9961.0,Chamounix South Fields,,Chamounix Drive,10081I,NaN,None,None,...,4.0,3.0,3.0,4.0,1.0,bac54543-b2e3-4f41-82fc-9aa638579897,1506614782385,maps.phl.data,1511536164731,David.Gould@phila.gov_phl


### Match using fuzzywuzzy

In [15]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [16]:
def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iterating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.token_sort_ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

In [17]:
def find_matches(x, right_data, score_cutoff):
    matches = process.extractBests(
        x, right_data, limit=1, score_cutoff=score_cutoff, scorer=fuzz.ratio
    )
    if len(matches): return matches[0]
    else: return np.nan


In [18]:
all_fuzzy_matches = []
cols = ["ASSET_NAME", "SITE_NAME", "ASSET_ADDR"]
registry = df.copy()
registry = registry.loc[~registry.index.isin(all_matches.index)]

for i, col in enumerate(cols):

    if col != "ASSET_ADDR":
        other_col = "Project site"
    else:
        
        
        other_col = "Address"

    matches = (
        registry[other_col]
        .apply(lambda x: find_matches(x, rebuild[col].dropna().str.strip(), score_cutoff=100))
        .dropna()
    )
    all_fuzzy_matches.append(matches)
    
all_fuzzy_matches = pd.concat(all_fuzzy_matches)

NameError: name 'all_matches' is not defined

In [43]:
import inspect

In [113]:
args = (1, 2, 3, 4)

In [142]:
def chained(f):
    def wrapper(*args, **kwargs):
        try:
            bound_args = inspect.signature(f).bind(*args, **kwargs)
            return f(*args, **kwargs)
        except:

            bound_args = inspect.signature(f).bind(*args[1:], **kwargs)
            merged, left, right = args[:3]
            matched = merged.dropna(subset=list(right.columns), how="all")
            left = left.loc[left.index.difference(matched.index)]
            new_matches = f(left, *args[2:], **kwargs)
            
            print(new_matches['right_index'].notnull().sum())
            return pd.concat([matched, new_matches], axis=0).sort_index()

    return wrapper

In [143]:
@chained
def merge(left, right, left_on, right_on, how="exact"):
    def contains(row, right):
        return right.loc[right[right_on].str.contains(row[left_on], na=False)]

    def exact(row, right):
        return right.loc[right[right_on] == row[left_on]]

    def startswith(row, right):
        return right.loc[right[right_on].str.startswith(row[left_on], na=False)]

    if how == "exact":
        comparison = exact
    elif how == "contains":
        comparison = contains
    elif how == "startswith":
        comparison = startswith

    # rename the index
    right = right.rename_axis("right_index").reset_index()

    merged = pd.concat(
        left.apply(
            lambda row: comparison(row, right).assign(index_left=row.name), axis=1
        ).tolist()
    )
    return left.merge(
        merged.set_index("index_left"), left_index=True, right_index=True, how="left"
    )

In [144]:
registry = rebuild.copy()

In [145]:
X = (
    merge(scraped, registry, left_on="Project site", right_on="SITE_NAME", how="exact")
    .pipe(
        merge,
        scraped,
        registry,
        left_on="Project site",
        right_on="SITE_NAME",
        how="contains",
    )
    .pipe(
        merge,
        scraped,
        registry,
        left_on="Project site",
        right_on="SITE_NAME",
        how="startswith",
    )
)

16
0


In [153]:
X.loc[X.index.duplicated(keep=False)]

,Project site,Address,Status,right_index,geometry,OBJECTID,ASSET_ID,ASSET_NAME,ASSET_ADDR,SITE_NAME,...,RebuildSitesFinalTable_20_14171,RebuildSitesFinalTable_20_14172,RebuildSitesFinalTable_20_14173,RebuildSitesFinalTable_20_14174,NC_PERMITS_CLASSES,GlobalID,CreationDate,Creator,EditDate,Editor
25,Cobbs Creek Park,280 Cobbs Creek Pkwy.,"Time-sensitive improvements, more to come",405.0,POINT (-75.24818842349863 39.95813417183293),194.0,983.0,Cobbs Creek Park and Recreation Center,280 COBBS CREEK PKWY,Cobbs Creek Park & Rec Center,...,5.0,4.0,1.0,1.0,1.0,c9250139-58d7-4630-8e78-6f23d504e0fc,1.506615e+12,maps.phl.data,1.528719e+12,David.Gould@phila.gov_phl
25,Cobbs Creek Park,280 Cobbs Creek Pkwy.,"Time-sensitive improvements, more to come",95.0,POINT (-75.25169780141226 39.96780318563707),105.0,995.0,Bocce Building,6600 VINE ST,Cobbs Creek Park Bocce,...,3.0,3.0,2.0,2.0,1.0,97f2a0e6-67d2-42c2-a293-d36080dc5dc6,1.506615e+12,maps.phl.data,1.531263e+12,David.Gould@phila.gov_phl
25,Cobbs Creek Park,280 Cobbs Creek Pkwy.,"Time-sensitive improvements, more to come",550.0,POINT (-75.24810739997631 39.95930110451342),551.0,984.0,Laura Simms Skatehouse / Cobbs Creek Ice Rink,12 COBBS CREEK PKWY,Cobbs Creek Park & Rec Center,...,5.0,4.0,1.0,1.0,1.0,4fd5d6aa-1213-451e-b3a5-0cd5d7a2c1c6,1.506615e+12,maps.phl.data,1.511910e+12,David.Gould@phila.gov_phl


In [ ]:
import multiprocessing


def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)


def _apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop("workers")
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(
        _apply_df, [(d, func, kwargs) for d in np.array_split(df, workers)]
    )
    pool.close()
    return pd.concat(list(result))


def _find_matches(x, right_data, score_cutoff):
    return process.extractBests(
        x, right_data, limit=10, score_cutoff=score_cutoff, scorer=fuzz.ratio
    )

In [25]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [26]:
left_on = "Project site"
right_on = "SITE_NAME"
score_cutoff = 90
workers = 4

In [27]:
def fuzzy_merge(left, right, left_on, right_on):

    # get the left and right strings
    left_data = left[left_on].dropna().astype(str)
    right_data = right[right_on].dropna().astype(str).rename_axis("right_index")

    # get the fuzzy matches
    fuzzy_matches = (
        _apply_by_multiprocessing(
            left_data,
            _find_matches,
            right_data=right_data,
            score_cutoff=score_cutoff,
            workers=workers,
        )
        .reindex(left.index)
        .rename_axis("left_index")
        .reset_index()
    )

    unstacked = (
        fuzzy_matches.apply(lambda x: pd.Series(x[left_data.name]), axis=1)
        .stack()
        .reset_index(level=1, drop=True)
    )

    matches = (
        fuzzy_matches.drop(left_data.name, axis=1)
        .join(
            pd.DataFrame(
                unstacked.tolist(),
                columns=[right_data.name, "score", "right_index"],
                index=unstacked.index,
            )
        )
        .set_index("left_index")
        .dropna()
    )

    matches = (
        pd.merge(
            left.loc[matches.index]
            .assign(
                right_index=matches.right_index.values,
                match_probability=matches.score.values / 100.0,
            )
            .rename_axis("left_index")
            .reset_index()
            .set_index("right_index"),
            right,
            left_index=True,
            right_index=True,
        )
        .rename_axis("right_index")
        .reset_index()
        .set_index("left_index")
        .sort_index()
    )

    return (
        pd.concat([matches, left.loc[left.index.difference(matches.index)]], sort=False)
        .sort_index()
        .loc[
            :,
            list(left.columns)
            + ["match_probability", "right_index"]
            + list(right.columns),
        ]
    )

In [28]:
Y = fuzzy_merge(scraped, registry, "Project site", "SITE_NAME")

NameError: name 'scraped' is not defined

In [258]:
Y

,Project site,Address,Status,match_probability,right_index,geometry,OBJECTID,ASSET_ID,ASSET_NAME,ASSET_ADDR,...,RebuildSitesFinalTable_20_14171,RebuildSitesFinalTable_20_14172,RebuildSitesFinalTable_20_14173,RebuildSitesFinalTable_20_14174,NC_PERMITS_CLASSES,GlobalID,CreationDate,Creator,EditDate,Editor
1,capitolo playground,900 federal st,Under construction,1.0,421.0,POINT (-75.1593818382852 39.93387898706955),366.0,501.0,capitolo playground,900 federal st,...,2.0,2.0,2.0,2.0,1.0,da4d00bd-2504-4c79-a4e7-afbea32dd81c,1.506615e+12,maps.phl.data,1.528719e+12,David.Gould@phila.gov_phl
2,cecil b moore recreation center,2551 n 22nd st,Community engagement and design,1.0,397.0,POINT (-75.16773310928932 39.99367232535786),128.0,1163.0,cecil b moore recreation center building,2020 50 w lehigh ave,...,4.0,4.0,1.0,1.0,1.0,4bf8b987-a931-427c-a3d4-f7472dbb4de4,1.506615e+12,maps.phl.data,1.528719e+12,David.Gould@phila.gov_phl
3,disston recreation center,4423 longshore ave,Under construction,1.0,398.0,POINT (-75.0438802250756 40.02842394749632),137.0,267.0,disston recreation center building 1,4423 longshore ave,...,3.0,2.0,2.0,4.0,1.0,39fdd8f5-110d-438a-b607-f5b97a314eb4,1.506615e+12,maps.phl.data,1.528719e+12,David.Gould@phila.gov_phl
4,east poplar,820 n 8th st,Community engagement and design,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,fishtown recreation center,1202 e montgomery ave,Under construction,0.9,398.0,POINT (-75.0438802250756 40.02842394749632),137.0,267.0,disston recreation center building 1,4423 longshore ave,...,3.0,2.0,2.0,4.0,1.0,39fdd8f5-110d-438a-b607-f5b97a314eb4,1.506615e+12,maps.phl.data,1.528719e+12,David.Gould@phila.gov_phl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,vogt recreation center,4131 unruh ave,"Time-sensitive repairs, more to come",1.0,431.0,POINT (-75.04921586048781 40.0277318195005),432.0,1.0,vogt recreation center building,4131 unruh ave,...,3.0,2.0,2.0,4.0,1.0,a60db4ac-9acc-4a5c-bf8a-ce2e4bfae5c7,1.506615e+12,maps.phl.data,1.528719e+12,David.Gould@phila.gov_phl
59,waterloo playground,2502 12 n howard st,Community engagement and design,1.0,415.0,POINT (-75.13292051672974 39.9882924189594),318.0,633.0,waterloo playground building,2502 12 n howard st,...,4.0,2.0,1.0,2.0,2.0,c2bc5cbb-347e-4b30-9068-c5ddbd70ad54,1.506615e+12,maps.phl.data,1.528719e+12,David.Gould@phila.gov_phl
60,west mill creek playground,5100 parrish st,"Time-sensitive repairs, more to come",1.0,472.0,POINT (-75.22255291270749 39.96660073329441),473.0,994.0,west mill creek playground building,5072 ogden st,...,4.0,3.0,1.0,1.0,1.0,5f2c03a6-fda5-420f-ae39-900b8e878a06,1.506615e+12,maps.phl.data,1.528719e+12,David.Gould@phila.gov_phl
61,wynnefield library,5325 overbrook ave,"Time-sensitive repairs, more to come",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [216]:
pd.merge(
    scraped.loc[Y.index]
    .assign(right_index=Y.right_index.values)
    .rename_axis("left_index")
    .reset_index()
    .set_index("right_index"),
    registry,
    left_index=True,
    right_index=True,
).rename_axis('right_index').reset_index().set_index('left_index').sort_index()

,right_index,Project site,Address,Status,geometry,OBJECTID,ASSET_ID,ASSET_NAME,ASSET_ADDR,SITE_NAME,...,RebuildSitesFinalTable_20_14171,RebuildSitesFinalTable_20_14172,RebuildSitesFinalTable_20_14173,RebuildSitesFinalTable_20_14174,NC_PERMITS_CLASSES,GlobalID,CreationDate,Creator,EditDate,Editor
left_index,,,,,,,,,,,,,,,,,,,,,
1,421.0,capitolo playground,900 federal st,Under construction,POINT (-75.1593818382852 39.93387898706955),366,501.0,capitolo playground,900 federal st,capitolo playground,...,2.0,2.0,2.0,2.0,1.0,da4d00bd-2504-4c79-a4e7-afbea32dd81c,1506614782385,maps.phl.data,1528718850432,David.Gould@phila.gov_phl
2,397.0,cecil b moore recreation center,2551 n 22nd st,Community engagement and design,POINT (-75.16773310928932 39.99367232535786),128,1163.0,cecil b moore recreation center building,2020 50 w lehigh ave,cecil b moore recreation center,...,4.0,4.0,1.0,1.0,1.0,4bf8b987-a931-427c-a3d4-f7472dbb4de4,1506614782385,maps.phl.data,1528718812481,David.Gould@phila.gov_phl
3,398.0,disston recreation center,4423 longshore ave,Under construction,POINT (-75.0438802250756 40.02842394749632),137,267.0,disston recreation center building 1,4423 longshore ave,disston recreation center,...,3.0,2.0,2.0,4.0,1.0,39fdd8f5-110d-438a-b607-f5b97a314eb4,1506614782385,maps.phl.data,1528718815167,David.Gould@phila.gov_phl
5,398.0,fishtown recreation center,1202 e montgomery ave,Under construction,POINT (-75.0438802250756 40.02842394749632),137,267.0,disston recreation center building 1,4423 longshore ave,disston recreation center,...,3.0,2.0,2.0,4.0,1.0,39fdd8f5-110d-438a-b607-f5b97a314eb4,1506614782385,maps.phl.data,1528718815167,David.Gould@phila.gov_phl
6,394.0,gifford playground,575 tomlinson rd,Under construction,POINT (-75.02499937506322 40.11443132550627),110,280.0,gifford playground building,575 tomlinson rd,gifford playground,...,3.0,2.0,2.0,3.0,1.0,3eb18a71-2eeb-48f2-afd7-d4a814e243a7,1506614782385,maps.phl.data,1528718808053,David.Gould@phila.gov_phl
8,120.0,hancock playground,147 master st,Community engagement and design,POINT (-75.13759602489284 39.9724196441212),136,1001.0,hancock st john s playground building,147 master st,hancock playground,...,3.0,2.0,2.0,2.0,2.0,01a68a8b-c4c3-4f59-92c9-841ee1b7124d,1506614782385,maps.phl.data,1528718814151,David.Gould@phila.gov_phl
9,400.0,heitzman recreation center,2136 castor ave,Community engagement and design,POINT (-75.09906508587883 39.99773639149318),143,1112.0,heitzman recreation center,2136 castor ave,heitzman recreation center,...,2.0,2.0,2.0,1.0,1.0,891fed28-845c-4a25-a85d-2c618e63e3fe,1506614782385,maps.phl.data,1528718818574,David.Gould@phila.gov_phl
10,411.0,miles mack playground,732 66 n 36th st,Community engagement and design,POINT (-75.19545801421876 39.96741455554081),282,482.0,miles mack playground,732 66 n 36th st,miles mack playground,...,4.0,5.0,1.0,2.0,1.0,40f160ba-0233-48f6-be70-63e762003a5e,1506614782385,maps.phl.data,1528718834537,David.Gould@phila.gov_phl
13,243.0,vare recreation center,2600 morris st,Community engagement and design,POINT (-75.08242590486071 40.04034749558613),272,197.0,tarken recreation center ice rink,6250 frontenac st,tarken recreation center,...,3.0,2.0,2.0,4.0,1.0,677dd411-7506-45d9-8a9f-4b7800916588,1506614782385,maps.phl.data,1511536038980,David.Gould@phila.gov_phl


In [203]:
Y.right_index

0     421.0
1     397.0
2     398.0
3       NaN
4     398.0
      ...  
57    431.0
58    415.0
59    472.0
60      NaN
61    502.0
Name: right_index, Length: 65, dtype: float64

In [214]:
pd.merge(
    scraped.loc[Y.index]
    .assign(right_index=Y.right_index.values)
    .rename_axis("left_index")
    .reset_index()
    .set_index("right_index"),
    registry,
    left_index=True,
    right_index=True,
).reset_index().set_index('left_index').sort_index()

,index,Project site,Address,Status,geometry,OBJECTID,ASSET_ID,ASSET_NAME,ASSET_ADDR,SITE_NAME,...,RebuildSitesFinalTable_20_14171,RebuildSitesFinalTable_20_14172,RebuildSitesFinalTable_20_14173,RebuildSitesFinalTable_20_14174,NC_PERMITS_CLASSES,GlobalID,CreationDate,Creator,EditDate,Editor
left_index,,,,,,,,,,,,,,,,,,,,,
1,421.0,capitolo playground,900 federal st,Under construction,POINT (-75.1593818382852 39.93387898706955),366,501.0,capitolo playground,900 federal st,capitolo playground,...,2.0,2.0,2.0,2.0,1.0,da4d00bd-2504-4c79-a4e7-afbea32dd81c,1506614782385,maps.phl.data,1528718850432,David.Gould@phila.gov_phl
2,397.0,cecil b moore recreation center,2551 n 22nd st,Community engagement and design,POINT (-75.16773310928932 39.99367232535786),128,1163.0,cecil b moore recreation center building,2020 50 w lehigh ave,cecil b moore recreation center,...,4.0,4.0,1.0,1.0,1.0,4bf8b987-a931-427c-a3d4-f7472dbb4de4,1506614782385,maps.phl.data,1528718812481,David.Gould@phila.gov_phl
3,398.0,disston recreation center,4423 longshore ave,Under construction,POINT (-75.0438802250756 40.02842394749632),137,267.0,disston recreation center building 1,4423 longshore ave,disston recreation center,...,3.0,2.0,2.0,4.0,1.0,39fdd8f5-110d-438a-b607-f5b97a314eb4,1506614782385,maps.phl.data,1528718815167,David.Gould@phila.gov_phl
5,398.0,fishtown recreation center,1202 e montgomery ave,Under construction,POINT (-75.0438802250756 40.02842394749632),137,267.0,disston recreation center building 1,4423 longshore ave,disston recreation center,...,3.0,2.0,2.0,4.0,1.0,39fdd8f5-110d-438a-b607-f5b97a314eb4,1506614782385,maps.phl.data,1528718815167,David.Gould@phila.gov_phl
6,394.0,gifford playground,575 tomlinson rd,Under construction,POINT (-75.02499937506322 40.11443132550627),110,280.0,gifford playground building,575 tomlinson rd,gifford playground,...,3.0,2.0,2.0,3.0,1.0,3eb18a71-2eeb-48f2-afd7-d4a814e243a7,1506614782385,maps.phl.data,1528718808053,David.Gould@phila.gov_phl
8,120.0,hancock playground,147 master st,Community engagement and design,POINT (-75.13759602489284 39.9724196441212),136,1001.0,hancock st john s playground building,147 master st,hancock playground,...,3.0,2.0,2.0,2.0,2.0,01a68a8b-c4c3-4f59-92c9-841ee1b7124d,1506614782385,maps.phl.data,1528718814151,David.Gould@phila.gov_phl
9,400.0,heitzman recreation center,2136 castor ave,Community engagement and design,POINT (-75.09906508587883 39.99773639149318),143,1112.0,heitzman recreation center,2136 castor ave,heitzman recreation center,...,2.0,2.0,2.0,1.0,1.0,891fed28-845c-4a25-a85d-2c618e63e3fe,1506614782385,maps.phl.data,1528718818574,David.Gould@phila.gov_phl
10,411.0,miles mack playground,732 66 n 36th st,Community engagement and design,POINT (-75.19545801421876 39.96741455554081),282,482.0,miles mack playground,732 66 n 36th st,miles mack playground,...,4.0,5.0,1.0,2.0,1.0,40f160ba-0233-48f6-be70-63e762003a5e,1506614782385,maps.phl.data,1528718834537,David.Gould@phila.gov_phl
13,243.0,vare recreation center,2600 morris st,Community engagement and design,POINT (-75.08242590486071 40.04034749558613),272,197.0,tarken recreation center ice rink,6250 frontenac st,tarken recreation center,...,3.0,2.0,2.0,4.0,1.0,677dd411-7506-45d9-8a9f-4b7800916588,1506614782385,maps.phl.data,1511536038980,David.Gould@phila.gov_phl


In [158]:
unstacked = (
    fuzzy_matches.apply(lambda x: pd.Series(x[left_data.name]), axis=1)
    .stack()
    .reset_index(level=1, drop=True)
)

matches = fuzzy_matches.drop(left_data.name, axis=1).join(
    pd.DataFrame(
        unstacked.tolist(),
        columns=[right_data.name, "score", right_data.index.name],
        index=unstacked.index,
    )
)

NameError: ("name 'left_data' is not defined", 'occurred at index 0')

In [23]:
import string


def clean_strings(df, cols, remove_punctuation=True, ignored=[]):
    """
    Clean the specified string columns in the input data frame.

    Parameters
    ----------
    df : DataFrame
        the input data 
    cols : list of str
        the list of column names; this should all be string types

    Notes
    -----
    This returns a copy of the input dataframe with cleaned versions of `cols`

    """
    # Check the inputs
    if not all(col in df.columns for col in cols):
        raise ValueError("Some of the input columns are not present in the data frame")
    if not all(hasattr(df[col], "str") for col in cols):
        raise ValueError("All specified columns should be of string types")

    # Return a copy
    out = df.copy()

    # Format each specified column
    for col in cols:

        # Make lower-cased and remove spaces
        X = out[col].dropna().str.lower().str.strip()

        # Remove punctuation
        if remove_punctuation:
            X = X.str.replace(f"[{string.punctuation}]", " ")

        # Remove ignored words
        out[col] = X.apply(
            lambda x: " ".join([w for w in x.split() if w not in ignored])
        ).str.strip()

    return out

In [64]:
x.apply(lambda x: " ".join([w for w in x.split() if w not in ['bad']]))

AttributeError: 'NoneType' object has no attribute 'split'

In [129]:
def find_exact_matches(x, right_data):
    
    a = x.lower().strip()
    b = right_data.str.lower().str.strip()
    
    matches = []
    exact = b == a
    if exact.sum():
        matches = right_data.index[exact]
    
    if not len(matches):
        semi_exact = b.str.startswith(a)
        matches = right_data.index[semi_exact]
        
    if len(matches) > 1:
        print(x, right_data.loc[matches])
    if len(matches):
        return matches[0]
    else:
        return np.nan
    

In [182]:
matched_indices = []
all_matches = []
cols = ["ASSET_NAME", "SITE_NAME", "ASSET_ADDR"]

for i, col in enumerate(cols):

    registry = df.copy()
    not_matched = ~df.index.isin(matched_indices)
    registry = registry.loc[not_matched]

    if col != "ASSET_ADDR":
        other_col = "Project site"
    else:
        other_col = "Address"

    matches = (
        registry[other_col]
        .apply(lambda x: find_exact_matches(x, rebuild[col].dropna()))
        .dropna()
    )

    matched_indices += matches.index.tolist()
    all_matches.append(matches)
    
all_matches = pd.concat(all_matches)

In [136]:
len(all_matches)

46

In [143]:
len(all_fuzzy_matches)

13

In [161]:
missing = (~df.index.isin(all_fuzzy_matches.index))&(~df.index.isin(all_matches))

In [162]:
(~df.index.isin(all_fuzzy_matches.index)).sum()

50

In [184]:
i = pd.Index(all_matches.index.tolist() + all_fuzzy_matches.index.tolist())


In [185]:
i[i.duplicated()]

Int64Index([], dtype='int64')

In [186]:
df.loc[~df.index.isin(all_matches.index.tolist() + all_fuzzy_matches.index.tolist())]

,Project site,Address,Status
26,Cobbs Creek Recreation Center,700 Cobbs Creek Pkwy.,"Time-sensitive repairs, more to come"
28,Fishtown Library,1217 E. Montgomery Ave.,"Time-sensitive repairs, more to come"
30,Fox Chase Recreation Center,7901 Ridgeway St.,"Time-sensitive repairs, more to come"
36,Kingsessing Library,1201 S. 51st St.,Awaiting nonprofit selection
38,Lawncrest Library,6098 Rising Sun Ave.,Community engagement and design
42,McPherson Square Library,601 E. Indiana Ave.,Awaiting nonprofit selection
43,McPherson Square Park,601 E. Indiana Ave.,Awaiting nonprofit selection
50,Parkside Evans Field,5300 Parkside Ave.,Under construction
54,Rivera Recreation Center / Mann Older Adult Ce...,601 E. Indiana Ave.,Under construction


In [165]:
df.loc[~missing]

,Project site,Address,Status
7,Glavin Playground,3267-81 Almond St.,Community engagement and design
11,Paschalville Library,6942 Woodland Ave.,Awaiting nonprofit selection
15,Al Pearlman Sports Complex,600 Port Royal Ave.,"Time-sensitive repairs, more to come"
19,Blanche A. Nixon Library,5800 Cobbs Creek Pkwy.,"Time-sensitive repairs, more to come"
26,Cobbs Creek Recreation Center,700 Cobbs Creek Pkwy.,"Time-sensitive repairs, more to come"
31,Francis J. Meyers Recreation Center,5800 Chester Ave.,"Time-sensitive repairs, more to come"
42,McPherson Square Library,601 E. Indiana Ave.,Awaiting nonprofit selection
43,McPherson Square Park,601 E. Indiana Ave.,Awaiting nonprofit selection
50,Parkside Evans Field,5300 Parkside Ave.,Under construction
51,Pelbano Recreation Center,8101 Bustleton Ave.,"Time-sensitive repairs, more to come"


In [216]:
dict_list = []
# match using site name 
for name in df['Project site']:
    match = match_name(name, rebuild.SITE_NAME, 90)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"match_name" : name})
    dict_.update({"SITE_NAME" : match[0]})
    dict_.update({"score" : match[1]})
#    dict.update({"geometry": rebuild['geometry']}) ### doesnt work 
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)


# create dataframe that still doesnt have matches 
site_n = merge_table.loc[merge_table['score'] == -1] 
# df2 goes into match function 
df2 = df.loc[df['Project site'].isin(site_n['match_name'])] 


# matches for first
site_m1 = merge_table.loc[merge_table['score'] != -1 ].sort_values('SITE_NAME').reset_index()  
# a is subset of rebuild dataframe for matched site names 
a = rebuild.loc[rebuild['SITE_NAME'].isin(site_m1['SITE_NAME'])].sort_values('SITE_NAME').reset_index()
# take geometry from a and put as column in matches 
site_m1['geometry'] = a['geometry'] 
# drop index and site_name so can concat to other matches later
site_m1 = site_m1.drop(['index', 'SITE_NAME'], axis = 1) 

In [220]:
dict_list = []
# match using asset name for sites that arent matched yet (df2)
for name in df2['Project site']:
    match = match_name(name, rebuild.ASSET_NAME, 85)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"match_name" : name})
    dict_.update({"ASSET_NAME" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)


# dataframe that still doesnt have matches 
asset_n = merge_table.loc[merge_table['score'] == -1] 
df3 = df2.loc[df2['Project site'].isin(asset_n['match_name'])] 

# matches for second 
site_m2 = merge_table.loc[merge_table['score'] != -1 ].sort_values('ASSET_NAME').reset_index()
a = rebuild.loc[rebuild['ASSET_NAME'].isin(site_m2['ASSET_NAME'])].sort_values('ASSET_NAME').reset_index()
site_m2['geometry'] = a['geometry']
site_m2 = site_m2.drop(['index', 'ASSET_NAME'], axis = 1)

In [268]:
dict_list = []
# match ussing address for sites that arent matched yet (df3)
for name in df3['Address']:
    match = match_name(name, rebuild.ASSET_ADDR, 90)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"match_name" : name})
    dict_.update({"ASSET_ADDR" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)

# dataframe that still does have matches 
adrs_n = merge_table.loc[merge_table['score'] == -1] 
adrs_n = df3.loc[df3['Address'].isin(adrs_n['match_name'])]  


# matches for third 
site_m3 = merge_table.loc[merge_table['score'] != -1].sort_values('ASSET_ADDR').drop_duplicates('ASSET_ADDR').reset_index()
a = rebuild.loc[rebuild['ASSET_ADDR'].isin(site_m3['ASSET_ADDR'])].sort_values('ASSET_ADDR').reset_index()
site_m3['geometry'] = a['geometry']
site_m3 = site_m3.drop(['index', 'ASSET_ADDR'], axis = 1)

In [271]:
match =  pd.concat([site_m1, site_m2, site_m3], axis=0)
match

,match_name,score,geometry
0,8th & Diamond Playground,100,POINT (-75.14745424859925 39.98337655477285)
1,Athletic Recreation Center,100,POINT (-75.17825478966257 39.97771982788517)
2,Barrett Playground,100,POINT (-75.13567280093206 40.03127365506787)
3,Belfield Recreation Center,100,POINT (-75.15633887665476 40.04160580121268)
4,Capitolo Playground,100,POINT (-75.1593818382852 39.93387898706955)
5,Carroll Park,100,POINT (-75.23672770704781 39.97102106031431)
6,Cecil B. Moore Recreation Center,100,POINT (-75.16773310928932 39.99367232535786)
7,Cherashore Playground,100,POINT (-75.13693943601245 40.03866370145978)
8,Chew Playground,100,POINT (-75.17402029466038 39.93822118220981)
9,Cobbs Creek Environmental Center,100,POINT (-75.2506190898427 39.95201743911613)


In [230]:
df

,Project site,Address,Status
1,Capitolo Playground,900 Federal St.,Under construction
2,Cecil B. Moore Recreation Center,2551 N. 22nd St.,Community engagement and design
3,Disston Recreation Center,4423 Longshore Ave.,Under construction
4,East Poplar,820 N. 8th St.,Community engagement and design
5,Fishtown Recreation Center,1202 E. Montgomery Ave.,Under construction
...,...,...,...
58,Vogt Recreation Center,4131 Unruh Ave.,"Time-sensitive repairs, more to come"
59,Waterloo Playground,2502-12 N. Howard St.,Community engagement and design
60,West Mill Creek Playground,5100 Parrish St.,"Time-sensitive repairs, more to come"
61,Wynnefield Library,5325 Overbrook Ave.,"Time-sensitive repairs, more to come"
